In [411]:
import pandas as pd

In [412]:
pd.set_option('display.max_columns', None)
#pd.reset_option('display.max_columns')

loading in prepared match and betting data

In [413]:
seasons_df=pd.read_pickle('data/seasons_betting_df.pkl')

loading in the data I collected. I have a table showing aggegated data for the previous season, a table showing all summer transfers and a table showing all winter transfers

In [414]:
len(set(seasons_df['HomeTeam']))

43

In [415]:
summerTransfers=pd.read_excel('data/SummerTransfers.xlsx')
winterTransfers=pd.read_excel('data/WinterTransfers.xlsx')
PLFinalStandings=pd.read_excel('data/PremierLeagueFinalTable.xlsx')

I need to rename 'AFC Bournemouth' to 'Bournemouth' so I can create a mapping more easily

In [416]:
summerTransfers['Club']=summerTransfers['Club'].replace({'AFC Bournemouth':'Bournemouth'})
winterTransfers['Club']=winterTransfers['Club'].replace({'AFC Bournemouth':'Bournemouth'})
PLFinalStandings['Club']=PLFinalStandings['Club'].replace({'AFC Bournemouth':'Bournemouth'})

In [417]:
#same mapping can be used for winter and summer transfer tables
home_teams = sorted(set(seasons_df['HomeTeam']))
clubs = sorted(set(summerTransfers['Club']))

mapping_df = pd.DataFrame({
    "HomeTeams": home_teams,
    "Clubs": clubs
})

teamTransferMap=mapping_df.set_index('Clubs')['HomeTeams'].to_dict()

In [418]:
summerTransfers['Club']=summerTransfers['Club'].map(teamTransferMap)
winterTransfers['Club']=winterTransfers['Club'].map(teamTransferMap)

In [419]:
len(home_teams)

43

In [420]:
home_teams = sorted(set(seasons_df['HomeTeam']))
home_teams.remove('Luton')  # Luton only appear in the final season, so no previous year's data
clubs = sorted(set(PLFinalStandings['Club']))


mapping_df = pd.DataFrame({
    "HomeTeams": home_teams,
    "Clubs": clubs
})

teamFinalStandingMap=mapping_df.set_index('Clubs')['HomeTeams'].to_dict()

In [421]:

PLFinalStandings['Club']=PLFinalStandings['Club'].map(teamFinalStandingMap)

### Preparing the transfer data for joining onto my features table

need to reformat the euro columns into ints

In [422]:
def convert_money(value):
    """
    Convert monetary values from string with 'k' or 'm' to numerical values.
    Handles both positive and negative values.
    If the value is '-', it is treated as 0.
    If the value is already an integer 0, it remains 0.
    """
    try:
        # If the value is an integer 0, leave it as 0
        if value == 0:
            return 0
        
        # If the value is a hyphen ('-'), treat it as 0
        if value == '-':
            return 0
        
        # If the value is already in string format but equivalent to 0, return 0
        if value == '0':
            return 0

        value = value.replace('€', '').strip()  # Remove currency symbol
        sign = -1 if value.startswith('-') else 1  # Detect if it's negative
        value = value.lstrip('-')  # Remove negative sign for processing
        
        if 'k' in value.lower():  # Handle 'k' for thousands
            return sign * float(value.replace('k', '').strip()) * 1_000
        elif 'm' in value.lower():  # Handle 'm' for millions
            return sign * float(value.replace('m', '').strip()) * 1_000_000
        else:
            return sign * float(value)  # No 'k' or 'm', return as float directly
    except ValueError as e:
        # Print the problematic value and error message
        print(f"Error converting value: {value}. Error: {e}")
        return None  # or return a default value if appropriate


In [423]:
summerTransfers[['Expenditure','Income','Balance']]=summerTransfers[['Expenditure','Income','Balance']].map(convert_money)
winterTransfers[['Expenditure','Income','Balance']]=winterTransfers[['Expenditure','Income','Balance']].map(convert_money)

In [424]:
winterTransfers

,Club,Expenditure,Arrivals,Income,Departures,Balance,Season
0,Arsenal,26500000.0,6,4250000.0,5,-22250000.0,season05
1,Man United,18500000.0,6,0.0,9,-18500000.0,season05
2,West Ham,11040000.0,7,150000.0,9,-10890000.0,season05
3,Portsmouth,9550000.0,12,0.0,14,-9550000.0,season05
4,Man City,8880000.0,6,0.0,3,-8880000.0,season05
...,...,...,...,...,...,...,...
375,Chelsea,0.0,5,7000000.0,5,7000000.0,season23
376,Newcastle,0.0,2,0.0,2,0.0,season23
377,Man United,0.0,3,3000000.0,7,3000000.0,season23
378,Bournemouth,0.0,3,0.0,6,0.0,season23


aggregating the total arrivals and depatures

In [425]:
summerTransfers['TotArrivals']=summerTransfers['Arrivals']-summerTransfers['Departures']
winterTransfers['TotArrivals']=winterTransfers['Arrivals']-winterTransfers['Departures']

only selecting aggregated columns for join

In [426]:
summerTransfers=summerTransfers[['Club','TotArrivals','Balance','Season']]
winterTransfers=winterTransfers[['Club','TotArrivals','Balance','Season']]

appending on the summer transfer data to my main features data set. I will only have this information available after august because that is when the summer transfer window closes

#### summer

In [427]:
seasons_df=seasons_df.merge(summerTransfers, how='left', left_on=['HomeTeam','season'], right_on=['Club','Season'])
seasons_df.drop(columns=['Club','Season'], inplace=True)
seasons_df=seasons_df.rename(columns={'TotArrivals':'home.totArrivals',
                           'Balance':'home.Balance'
                          }
                 )

In [428]:
seasons_df=seasons_df.merge(summerTransfers, how='left', left_on=['AwayTeam','season'], right_on=['Club','Season'])
seasons_df.drop(columns=['Club','Season'], inplace=True)
seasons_df=seasons_df.rename(columns={'TotArrivals':'away.totArrivals',
                           'Balance':'away.Balance'
                          }
                 )

getting the diff of the home and away teams summer transfer activity, and dropping the individual data

In [429]:
seasons_df['diff.totSummerArrivals']=seasons_df['home.totArrivals']-seasons_df['away.totArrivals']
seasons_df.drop(columns=['home.totArrivals','away.totArrivals'], inplace=True)

In [430]:
seasons_df['diff.totSummerBalance']=seasons_df['away.Balance']-seasons_df['home.Balance']
seasons_df.drop(columns=['home.Balance','away.Balance'], inplace=True)

overwriting the summer transfer data with 0 if we are still in August

In [431]:

seasons_df.loc[seasons_df['Date'].dt.month==8, 'diff.totSummerArrivals']=0
seasons_df.loc[seasons_df['Date'].dt.month==8, 'diff.totSummerBalance']=0

appending on the summer transfer data to my main features data set. I will only have this information available after January because that is when the winter transfer window ends.

#### winter

In [432]:
seasons_df=seasons_df.merge(winterTransfers, how='left', left_on=['HomeTeam','season'], right_on=['Club','Season'])
seasons_df.drop(columns=['Club','Season'], inplace=True)
seasons_df=seasons_df.rename(columns={'TotArrivals':'home.totArrivals',
                           'Balance':'home.Balance'
                          }
                 )

In [433]:
seasons_df=seasons_df.merge(winterTransfers, how='left', left_on=['AwayTeam','season'], right_on=['Club','Season'])
seasons_df.drop(columns=['Club','Season'], inplace=True)
seasons_df=seasons_df.rename(columns={'TotArrivals':'away.totArrivals',
                           'Balance':'away.Balance'
                          }
                 )

In [434]:
seasons_df['diff.totWinterArrivals']=seasons_df['home.totArrivals']-seasons_df['away.totArrivals']
seasons_df.drop(columns=['home.totArrivals','away.totArrivals'], inplace=True)

In [435]:
seasons_df['diff.totWinterBalance']=seasons_df['away.Balance']-seasons_df['home.Balance']
seasons_df.drop(columns=['home.Balance','away.Balance'], inplace=True)

overwriting the summer transfer data unless we are in Februrary, March, April, May, June or July (the covid season ran until July)

In [436]:
seasons_df.loc[seasons_df['Date'].dt.month.isin([1,8,9,10,11,12]), 'diff.totWinterArrivals']=0
seasons_df.loc[seasons_df['Date'].dt.month.isin([1,8,9,10,11,12]), 'diff.totWinterBalance']=0

In [437]:
summerTransfers

,Club,TotArrivals,Balance,Season
0,Chelsea,-2,-52900000.0,season05
1,Newcastle,-2,-28600000.0,season05
2,Liverpool,1,-16410000.0,season05
3,Everton,1,-32730000.0,season05
4,Tottenham,2,-12950000.0,season05
...,...,...,...,...
375,Brentford,1,-54600000.0,season23
376,Sheffield United,5,-33550000.0,season23
377,Everton,0,42300000.0,season23
378,Crystal Palace,-2,-38700000.0,season23


### Preparing the PLFinalStandings data for joining onto my features table

I want to the position the team finished in the previous season. Goals scored and goals conceded for the previous year will also be interesting. But it will be a bit of a subjective decision on how to fill in that data for the 3 promoted teams each year. 

Perhaps I can find the data from their championship finishing season and then scale down the goals scored and scale up the goals conceded.

For now I will just focus on the finishing positions of the previous season

Not all teams will have a finishing position from the previous year (the 3 promoted teams). So I need to think of a way to capture that.

I will flip the premier league table and award 20 points for finishing first, and 1 point for finishing last. The 3 promoted teams will have a null after the join and I will fill with 0.

In [438]:
PLFinalStandings['PositionAward']=21-PLFinalStandings['Position']

Only focus on the final position for now

In [439]:
PLFinalStandings=PLFinalStandings[['Club','Season','PositionAward']]

Now I want to change all season values to increase by 1 (season08 --> season09) so i can do a simple join

In [440]:
PLFinalStandings['Season']=PLFinalStandings['Season'].apply(lambda x: 'season' + str(int(x[-2:]) + 1).zfill(2))

In [441]:
seasons_df=seasons_df.merge(PLFinalStandings, how='left', left_on=['HomeTeam','season'], right_on=['Club','Season'])
seasons_df.drop(columns=['Club','Season'], inplace=True)
seasons_df=seasons_df.rename(columns={'PositionAward':'home.PositionAward'})

In [442]:
seasons_df=seasons_df.merge(PLFinalStandings, how='left', left_on=['AwayTeam','season'], right_on=['Club','Season'])
seasons_df.drop(columns=['Club','Season'], inplace=True)
seasons_df=seasons_df.rename(columns={'PositionAward':'away.PositionAward'})

filling promoted teams with a positionaward of 0

In [443]:
seasons_df['home.PositionAward']=seasons_df['home.PositionAward'].fillna(0)
seasons_df['away.PositionAward']=seasons_df['away.PositionAward'].fillna(0)

In [444]:
seasons_df['diff.PositionAward']=seasons_df['home.PositionAward']-seasons_df['away.PositionAward']
seasons_df.drop(columns=['home.PositionAward','away.PositionAward'], inplace=True)

##### getting the season 2 years previous. shift the season value by another year

In [445]:
PLFinalStandings['Season']=PLFinalStandings['Season'].apply(lambda x: 'season' + str(int(x[-2:]) + 1).zfill(2))

In [446]:
seasons_df=seasons_df.merge(PLFinalStandings, how='left', left_on=['HomeTeam','season'], right_on=['Club','Season'])
seasons_df.drop(columns=['Club','Season'], inplace=True)
seasons_df=seasons_df.rename(columns={'PositionAward':'home.PositionAward2'})

In [447]:
seasons_df=seasons_df.merge(PLFinalStandings, how='left', left_on=['AwayTeam','season'], right_on=['Club','Season'])
seasons_df.drop(columns=['Club','Season'], inplace=True)
seasons_df=seasons_df.rename(columns={'PositionAward':'away.PositionAward2'})

In [448]:
seasons_df['home.PositionAward2']=seasons_df['home.PositionAward2'].fillna(0)
seasons_df['away.PositionAward2']=seasons_df['away.PositionAward2'].fillna(0)

In [449]:
seasons_df['diff.PositionAward2']=seasons_df['home.PositionAward2']-seasons_df['away.PositionAward2']
seasons_df.drop(columns=['home.PositionAward2','away.PositionAward2'], inplace=True)

In [450]:
seasons_df.to_pickle('data/seasons_betting_constants_df.pkl')

In [451]:
seasons_df

,Date,HomeTeam,AwayTeam,FTR,home.matchNumber,away.matchNumber,season,B365H,B365D,B365A,MaxH,AvgH,MaxD,AvgD,MaxA,AvgA,diff.daysSince,diff.total.win,diff.total.draw,diff.total.loss,diff.total.goals,diff.total.conceded,diff.total.half_goals,diff.total.half_conceded,diff.total.shots,diff.total.shots_against,diff.total.shots_target,diff.total.shots_against_target,diff.total.corners,diff.total.corners_conceded,diff.total.fouls,diff.total.fouls_other_team,diff.total.yellows,diff.total.reds,diff.last3.win,diff.last3.draw,diff.last3.loss,diff.last3.goals,diff.last3.conceded,diff.last3.half_goals,diff.last3.half_conceded,diff.last3.shots,diff.last3.shots_against,diff.last3.shots_target,diff.last3.shots_against_target,diff.last3.corners,diff.last3.corners_conceded,diff.last3.fouls,diff.last3.fouls_other_team,diff.last3.yellows,diff.last3.reds,diff.total.win.whenHorA,diff.total.draw.whenHorA,diff.total.loss.whenHorA,diff.totSummerArrivals,diff.totSummerBalance,diff.totWinterArrivals,diff.totWinterBalance,diff.PositionAward,diff.PositionAward2
0,2005-08-13,Aston Villa,Bolton,D,1,1,season05,2.30,3.25,3.00,2.40,2.20,3.30,3.16,3.40,3.05,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,0,0.0,0,0.0,-4.0,2.0
1,2005-08-13,Everton,Man United,A,1,1,season05,5.00,3.40,1.72,5.65,4.69,3.70,3.36,1.80,1.69,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,0,0.0,0,0.0,-1.0,-14.0
2,2005-08-13,Fulham,Birmingham,D,1,1,season05,2.37,3.25,2.87,2.60,2.31,3.30,3.16,3.05,2.87,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,0,0.0,0,0.0,-1.0,1.0
3,2005-08-13,Man City,West Brom,D,1,1,season05,1.72,3.40,5.00,1.80,1.69,3.63,3.38,5.60,4.79,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,0,0.0,0,0.0,9.0,5.0
4,2005-08-13,Middlesbrough,Liverpool,D,1,1,season05,2.87,3.20,2.40,3.30,2.81,3.35,3.17,2.50,2.35,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,0,0.0,0,0.0,-2.0,-7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7215,2024-05-19,Crystal Palace,Aston Villa,H,38,38,season23,1.85,4.33,3.60,1.93,1.87,4.50,4.28,3.84,3.67,2.0,-0.216216,0.054054,0.162162,-0.648649,0.054054,-0.378378,0.0,-1.756757,0.324324,-0.756757,-0.513514,-1.432432,0.945946,1.108108,-1.108108,-0.594595,0.0,0.666667,-0.333333,-0.333333,1.0,-1.333333,0.333333,-0.666667,5.0,-7.0,2.333333,-3.0,2.666667,0.333333,1.666667,-3.333333,0.0,0.333333,-0.055556,0.0,0.055556,-3,-21750000.0,1,10600000.0,-4.0,2.0
7216,2024-05-19,Liverpool,Wolves,H,38,38,season23,1.17,8.00,15.00,1.18,1.16,10.00,8.71,17.00,14.55,-2.0,0.27027,0.081081,-0.351351,0.918919,-0.594595,0.324324,-0.189189,8.945946,-3.432432,2.891892,-1.432432,3.27027,-1.756757,-0.486486,-0.648649,-0.891892,0.054054,0.0,0.666667,-0.666667,1.666667,-0.666667,1.0,-1.0,13.0,-0.333333,5.0,-0.666667,2.666667,0.0,-1.666667,0.333333,-2.0,0.0,0.5,-0.055556,-0.444444,1,186500000.0,3,0.0,8.0,8.0
7217,2024-05-19,Luton,Fulham,A,38,38,season23,2.90,3.90,2.20,3.00,2.92,4.35,3.95,2.30,2.23,0.0,-0.162162,0.0,0.162162,-0.027027,0.594595,-0.027027,0.189189,-1.864865,3.567568,-0.972973,0.945946,-0.216216,1.0,1.72973,2.135135,-0.189189,-0.108108,0.0,-0.333333,0.333333,0.666667,0.333333,0.666667,0.333333,2.0,3.333333,2.0,0.0,2.0,-1.666667,5.666667,3.666667,1.0,-0.333333,0.055556,-0.11